# Text into Data

DS 5001 Text as Data

## Purpose

Ww import a text using the  **Clip, Chunk, and Split pattern**.

Demonstrate how to tokenize a raw text and map an OHCO onto the resulting dataframe of tokens.

This goes beyond what we did last week in the First Foray notebook. We capture the chapter, paragraph, and sentence structure of the text.

## Set Up

## Import libraries

In [1]:
import pandas as pd

### Import Config

In [2]:
import configparser
config = configparser.ConfigParser()
config.read("../../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']

In [3]:
data_home, output_dir

('/sfs/gpfs/tardis/home/rca2t/Documents/MSDS/DS5001/DS5001-2025-01-R/data',
 '/sfs/gpfs/tardis/home/rca2t/Documents/MSDS/DS5001/DS5001-2025-01-R/output')

In [4]:
text_file = f"{data_home}/gutenberg/pg105.txt"
csv_file  = f"{output_dir}/austen-persuasion.csv" # The file we will create

In [5]:
OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']

## Import file into a dataframe

In [6]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.replace(r'\n+', ' ', regex=True).str.strip()

In [7]:
LINES.sample(20)

,line_str
line_num,
3988,"satisfaction in the house in Camden Place, whi..."
6636,"difficulty or a scruple was there on his side,..."
8276,"they came into contact again, in seeing Anne r..."
2678,"they did not wish, and admired again the sort ..."
6017,novelty and beauty! I have travelled so little...
7939,"join or to pass on, said nothing, only looked...."
6460,
8705,Please check the Project Gutenberg Web pages f...
128,"discontented when a woman does marry again, th..."


## Extract Title 

In [8]:
title = LINES.loc[0].line_str.replace('The Project Gutenberg EBook of ', '')

In [9]:
print(title)

Persuasion, by Jane Austen


## Clip the Cruft

In [10]:
clip_pats = [
    r"\*\*\*\s*START OF (?:THE|THIS) PROJECT",
    r"\*\*\*\s*END OF (?:THE|THIS) PROJECT"
]

In [11]:
pat_a = LINES.line_str.str.match(clip_pats[0])
pat_b = LINES.line_str.str.match(clip_pats[1])

In [12]:
line_a = LINES.loc[pat_a].index[0] + 1
line_b = LINES.loc[pat_b].index[0] - 1

In [13]:
line_a, line_b

(19, 8372)

In [14]:
LINES = LINES.loc[line_a : line_b]

In [15]:
LINES.head(10)

,line_str
line_num,
19,
20,
21,
22,
23,Produced by Sharon Partridge and Martin Ward. ...
24,by Al Haines.
25,
26,
27,


In [16]:
LINES.tail(10)

,line_str
line_num,
8363,
8364,
8365,
8366,
8367,
8368,
8369,
8370,
8371,End of the Project Gutenberg EBook of Persuasi...


## Chunk by Chapter

### Find all chapter headers

The regex will depend on the source text. You need to investigate the source text to figure this out.

In [17]:
chap_pat = r"^\s*(?:chapter|letter)\s+\d+"

In [18]:
chap_lines = LINES.line_str.str.match(chap_pat, case=False) # Returns a truth vector

In [19]:
LINES.loc[chap_lines] # Use as filter for dataframe

,line_str
line_num,
47,Chapter 1
306,Chapter 2
500,Chapter 3
786,Chapter 4
959,Chapter 5
1297,Chapter 6
1657,Chapter 7
1992,Chapter 8
2346,Chapter 9


### Assign numbers to chapters

In [20]:
LINES.loc[chap_lines, 'chap_num'] = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]

In [21]:
LINES.loc[chap_lines]

,line_str,chap_num
line_num,,
47,Chapter 1,1.0
306,Chapter 2,2.0
500,Chapter 3,3.0
786,Chapter 4,4.0
959,Chapter 5,5.0
1297,Chapter 6,6.0
1657,Chapter 7,7.0
1992,Chapter 8,8.0
2346,Chapter 9,9.0


Notice that all lines that are not chapter headers have no chapter number assigned to them.

In [22]:
LINES.sample(10)

,line_str,chap_num
line_num,,
3131,Wentworth turned in to call on his friend; the...,NaN
8277,"seniority, and the mistress of a very pretty l...",NaN
1190,"""My dear Mary, recollect what a comfortable ac...",NaN
5019,"could use my hands she taught me to knit, whic...",NaN
7129,"been ostentatiously dropped by Mary, as well a...",NaN
3733,"Charles agreed, but declared his resolution of...",NaN
3790,"ill-judging claims of Mary; but so it must be,...",NaN
875,"solicited, when about two-and-twenty, to chang...",NaN
5600,Admiral interrupted her with--,NaN


### Forward-fill chapter numbers to following text lines

`ffill()` will replace null values with the previous non-null value.

In [23]:
LINES.chap_num = LINES.chap_num.ffill()

In [24]:
LINES.sample(10)

,line_str,chap_num
line_num,,
6102,"""This,"" said she, ""is nearly the sense, or rat...",20.0
7015,opportunity of bringing him and Sir Walter tog...,22.0
223,inferior birth.,1.0
4151,,14.0
935,the better feelings of her calmness; but the g...,4.0
318,much serious consideration. She was a woman r...,2.0
4612,must give him leave to present him to his youn...,15.0
4229,beauty.' Oh! there was no end of Miss Elliot's...,14.0
5809,were principally such as were wont to be alway...,19.0


Notice that the lines taht precede our first chapter have no chapters, which is what we want. We need to decide whether to keep these lines as textual front matter or to dispose of them.

In [25]:
LINES.head(20)

,line_str,chap_num
line_num,,
19,,NaN
20,,NaN
21,,NaN
22,,NaN
23,Produced by Sharon Partridge and Martin Ward. ...,NaN
24,by Al Haines.,NaN
25,,NaN
26,,NaN
27,,NaN


### Clean up

In [26]:
LINES = LINES.dropna(subset=['chap_num']) # Remove everything before Chapter 1
# LINES = LINES.loc[~LINES.chap_num.isna()] # Remove everything before Chapter 1 (alternate method)
LINES = LINES.loc[~chap_lines] # Remove chapter heading lines; their work is done
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

In [27]:
LINES.sample(10)

,line_str,chap_num
line_num,,
8098,not enough to make the fool of me which I appe...,23
5912,Captain Wentworth walked in alone. Anne was t...,20
4438,over all the others which they had either seen...,15
7134,intelligible account of the whole; a narration...,22
2960,and quietly obliged her to be assisted into th...,10
3308,"serious reflection, that, like many other grea...",11
3912,before to Mrs Musgrove and Henrietta; but with...,13
7930,"momentary vexation occurred. Charles, in his ...",23
1902,"Mary talked, but she could not attend. She ha...",7


### Group lines into chapters

In [28]:
OHCO[:1]

['chap_num']

In [29]:
# Make big string for each chapter
CHAPS = LINES.groupby(OHCO[:1])\
    .line_str.apply(lambda x: '\n'.join(x))\
    .to_frame('chap_str')

In [30]:
CHAPS.head(10)

,chap_str
chap_num,
1,"\n\nSir Walter Elliot, of Kellynch Hall, in So..."
2,"\n\nMr Shepherd, a civil, cautious lawyer, who..."
3,"\n\n""I must take leave to observe, Sir Walter,..."
4,"\n\nHe was not Mr Wentworth, the former curate..."
5,\n\nOn the morning appointed for Admiral and M...
6,\n\nAnne had not wanted this visit to Uppercro...
7,"\n\nA very few days more, and Captain Wentwort..."
8,\n\nFrom this time Captain Wentworth and Anne ...
9,\n\nCaptain Wentworth was come to Kellynch as ...


In [31]:
CHAPS['chap_str'] = CHAPS.chap_str.str.strip()

In [32]:
CHAPS

,chap_str
chap_num,
1,"Sir Walter Elliot, of Kellynch Hall, in Somers..."
2,"Mr Shepherd, a civil, cautious lawyer, who, wh..."
3,"""I must take leave to observe, Sir Walter,"" sa..."
4,"He was not Mr Wentworth, the former curate of ..."
5,On the morning appointed for Admiral and Mrs C...
6,"Anne had not wanted this visit to Uppercross, ..."
7,"A very few days more, and Captain Wentworth wa..."
8,From this time Captain Wentworth and Anne Elli...
9,Captain Wentworth was come to Kellynch as to a...


So, now we have our text grouped by chapters.

## Split chapters into paragraphs 

We use Pandas' convenient `.split()` method with `expand=True`, followed by `.stack()`.
Note that this creates zero-based indexes.

In [33]:
para_pat = r'\n\n+'

In [34]:
# CHAPS['chap_str'].str.split(para_pat, expand=True).head()

In [35]:
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[:2]

In [36]:
PARAS.head()

para_str
chap_num para_num                                                   
1        0         Sir Walter Elliot, of Kellynch Hall, in Somers...
         1                                 "ELLIOT OF KELLYNCH HALL.
         2         "Walter Elliot, born March 1, 1760, married, J...
         3         Precisely such had the paragraph originally st...
         4         Then followed the history and rise of the anci...

In [37]:
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True)
PARAS['para_str'] = PARAS['para_str'].str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs

In [38]:
PARAS.head()

para_str
chap_num para_num                                                   
1        0         Sir Walter Elliot, of Kellynch Hall, in Somers...
         1                                 "ELLIOT OF KELLYNCH HALL.
         2         "Walter Elliot, born March 1, 1760, married, J...
         3         Precisely such had the paragraph originally st...
         4         Then followed the history and rise of the anci...

## Split paragraphs into sentences

In [39]:
# sent_pat = r'[.?!;:"]+'
sent_pat = r'[.?!;:]+'
SENTS = PARAS['para_str'].str.split(sent_pat, expand=True).stack()\
    .to_frame('sent_str')
SENTS.index.names = OHCO[:3]

In [40]:
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip() # CRUCIAL TO REMOVE BLANK TOKENS

In [41]:
SENTS.head()

sent_str
chap_num para_num sent_num                                                   
1        0        0         Sir Walter Elliot, of Kellynch Hall, in Somers...
                  1         there he found occupation for an idle hour, an...
                  2         there his faculties were roused into admiratio...
                  3         there any unwelcome sensations, arising from d...
                  4         and there, if every other leaf were powerless,...

In [42]:
SENTS.sample(10)

,,,sent_str
chap_num,para_num,sent_num,
12,55,13,"Between these two, she could want no possible ..."
20,12,1,I could not leave it till Louisa's doing well ...
16,11,11,""""
4,9,4,She could do justice to the superiority of Lad...
20,7,1,"""but there, I think, ends the resemblance"
21,32,4,But I have not known him long
22,47,1,It would be unpardonable to fail
17,16,1,""" asked Elizabeth"
11,2,1,those rooms had witnessed former meetings whic...


## Split sentences into tokens

In [43]:
token_pat = r"[\s',-]+"
TOKENS = SENTS['sent_str'].str.split(token_pat, expand=True).stack()\
    .to_frame('token_str')

In [44]:
TOKENS.index.names = OHCO[:4]

In [45]:
TOKENS

token_str
chap_num para_num sent_num token_num            
1        0        0        0                 Sir
                           1              Walter
                           2              Elliot
                           3                  of
                           4            Kellynch
...                                          ...
24       13       0        6                  of
                           7          Persuasion
                           8                  by
                           9                Jane
                           10             Austen

[85014 rows x 1 columns]

## Extract Vocabulary

In [46]:
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
VOCAB = TOKENS.term_str.value_counts().to_frame('n').reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

In [47]:
VOCAB

,term_str,n
term_id,,
0,the,3330
1,to,2808
2,and,2800
3,of,2572
4,a,1595
...,...,...
5755,reins,1
5756,judiciously,1
5757,rut,1


## Gathering by Content Object

In [48]:
def gather(ohco_level):
    global TOKENS
    level_name = OHCO[ohco_level-1].split('_')[0]
    df = TOKENS.groupby(OHCO[:ohco_level])\
        .token_str.apply(lambda x: x.str.cat(sep=' '))\
        .to_frame(f"{level_name}_str")
    return df

In [49]:
gather(1)

,chap_str
chap_num,
1,Sir Walter Elliot of Kellynch Hall in Somerset...
2,Mr Shepherd a civil cautious lawyer who whatev...
3,"""I must take leave to observe Sir Walter "" sai..."
4,He was not Mr Wentworth the former curate of M...
5,On the morning appointed for Admiral and Mrs C...
6,Anne had not wanted this visit to Uppercross t...
7,A very few days more and Captain Wentworth was...
8,From this time Captain Wentworth and Anne Elli...
9,Captain Wentworth was come to Kellynch as to a...


In [50]:
gather(2)

para_str
chap_num para_num                                                   
1        0         Sir Walter Elliot of Kellynch Hall in Somerset...
         1                                  "ELLIOT OF KELLYNCH HALL
         2         "Walter Elliot born March 1 1760 married July ...
         3         Precisely such had the paragraph originally st...
         4         Then followed the history and rise of the anci...
...                                                              ...
24       9         Anne satisfied at a very early period of Lady ...
         10        Her recent good offices by Anne had been enoug...
         11        Mrs Smith s enjoyments were not spoiled by thi...
         12                                                    Finis
         13        End of the Project Gutenberg EBook of Persuasi...

[1008 rows x 1 columns]

In [51]:
gather(3)

sent_str
chap_num para_num sent_num                                                   
1        0        0         Sir Walter Elliot of Kellynch Hall in Somerset...
                  1         there he found occupation for an idle hour and...
                  2         there his faculties were roused into admiratio...
                  3         there any unwelcome sensations arising from do...
                  4         and there if every other leaf were powerless h...
...                                                                       ...
24       11       4         Anne was tenderness itself and she had the ful...
                  5         His profession was all that could ever make he...
                  6         She gloried in being a sailor s wife but she m...
         12       0                                                     Finis
         13       0         End of the Project Gutenberg EBook of Persuasi...

[5612 rows x 1 columns]

## Save work to CSV

This is important -- will be used for homework.

In [52]:
TOKENS.to_csv(csv_file)